In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from ast import literal_eval
import sklearn as sk
import seaborn as sb
import re, string
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")

import joblib

import torch
#import torchvision

In [110]:
data = pd.read_csv("preprocessed_data.csv")
data = data.dropna()

data.gender= data.gender.astype(str)

target = "gender"
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(data["text"], data[target], test_size=test_size,shuffle=True)

In [111]:
target = "sign"
test_size = 0.2

X_trainS, X_testS, y_trainS, y_testS = train_test_split(data["text"], data[target], test_size=test_size,shuffle=True)

In [112]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import LinearSVC


word_vec_pipe = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),])

lr_gender = LogisticRegression(solver="saga", penalty="l2", C=1)
lr_sign = LogisticRegression(solver="saga", penalty="l1", C=10)

estimators = ("rf", RandomForestClassifier(n_estimators=15, random_state=42)), ("svr", LinearSVC(random_state=42))

clf_gender = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(dual= False, max_iter= 100, penalty= 'l2'))
clf_sign = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(dual= False, max_iter= 100, penalty= 'l2'))

pipe_lr_gender = Pipeline([('prevec',word_vec_pipe),('model', lr_gender)])
pipe_clf_gender = Pipeline([('prevec',word_vec_pipe),('model', clf_gender)])

pipe_lr_sign = Pipeline([('prevec',word_vec_pipe),('model', lr_sign)])
pipe_clf_sign = Pipeline([('prevec',word_vec_pipe),('model', clf_sign)])

In [113]:
pipe_lr_gender.fit(X_train, y_train)
pipe_clf_gender.fit(X_train, y_train)

Pipeline(steps=[('prevec',
                 Pipeline(steps=[('vect', CountVectorizer()),
                                 ('tfidf', TfidfTransformer())])),
                ('model',
                 StackingClassifier(estimators=(('rf',
                                                 RandomForestClassifier(n_estimators=15,
                                                                        random_state=42)),
                                                ('svr',
                                                 LinearSVC(random_state=42))),
                                    final_estimator=LogisticRegression()))])

In [114]:
y_pred_lr_gender = pipe_lr_gender.predict(X_test)
y_pred_clf_gender = pipe_clf_gender.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred_lr_gender, y_test))
print('accuracy %s' % accuracy_score(y_pred_clf_gender, y_test))

accuracy 0.6796614516542703
accuracy 0.6753013593229034


In [81]:
pipe_lr_sign.fit(X_trainS, y_trainS)
pipe_clf_sign.fit(X_trainS, y_trainS)

Pipeline(steps=[('prevec',
                 Pipeline(steps=[('vect', CountVectorizer()),
                                 ('tfidf', TfidfTransformer())])),
                ('model',
                 StackingClassifier(estimators=(('rf',
                                                 RandomForestClassifier(n_estimators=15,
                                                                        random_state=42)),
                                                ('svr',
                                                 LinearSVC(random_state=42))),
                                    final_estimator=LogisticRegression()))])

In [82]:
y_pred_lr_sign = pipe_lr_sign.predict(X_testS)
y_pred_clf_sign = pipe_clf_sign.predict(X_testS)

print('accuracy %s' % accuracy_score(y_pred_lr_sign, y_testS))
print('accuracy %s' % accuracy_score(y_pred_clf_sign, y_testS))

accuracy 0.1949217748140549
accuracy 0.21261862015901514


In [120]:
filename_1 = 'gender_log_reg.joblib'
filename_2 = 'gender_stacked.joblib'
filename_3 = 'sign_log_reg.joblib'
filename_4 = 'sign_stacked.joblib'
#pickle.dump(lr, open(filename_1, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
#pickle.dump(clf, open(filename_2, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
joblib.dump(pipe_lr_gender, filename_1)
joblib.dump(pipe_clf_gender, filename_2)
#joblib.dump(pipe_lr_sign, filename_3)
#joblib.dump(pipe_clf_sign, filename_4)

['gender_stacked.joblib']

In [ ]:
loaded_model = joblib.load(filename_1)
result = loaded_model.score(X_test, y_test)
print(result)

0.6823076923076923
